In [1]:
from collections import Counter
import pandas as pd

In [2]:
with open('./DATASET/train/truthful.txt') as t, open('./DATASET/train/deceptive.txt') as d:
    truthful = t.read()
    deceptive = d.read()

In [3]:
def add_start_characters(words):
    words = '<s> ' + words
    words = words.replace('\n', ' <s> ')
    return words[:-5]

## Unigrams

In [4]:
truthful_unigram = dict(Counter(add_start_characters(truthful).split()))
deceptive_unigram = dict(Counter(add_start_characters(deceptive).split()))

In [150]:
def get_unigram_prob(corpus, unigram_key):
    total_words = 0
    for key in corpus:
        total_words += corpus[key]

    return corpus.get(unigram_key, 0)/total_words

In [151]:
get_unigram_prob(deceptive_unigram, 'I')

0.02905073649754501

## Unsmoothed Bigrams

In [98]:
def get_bigram_counts(word_list):
    corpus = {}
    for i, word in enumerate(word_list[1:], start=1):
        if word != '<s>':
            if (word_list[i-1], word) not in corpus:
                corpus[(word_list[i-1], word)] = 1
            else:
                corpus[(word_list[i-1], word)] += 1
        
    return corpus

In [99]:
truthful_bigram = get_bigram_counts(add_start_characters(truthful).split())
deceptive_bigram = get_bigram_counts(add_start_characters(deceptive).split())

In [100]:
def get_bigram_prob(corpus, bigram_to_test):
    total_words = 0
    for key in corpus:
        if key[0] == bigram_to_test[0]:
            total_words += corpus[key]
    return corpus.get(bigram_to_test, 0)/total_words

In [101]:
get_bigram_prob(deceptive_bigram, ('I', 'am'))

0.01488933601609658

### Smoothed Bigrams

In [102]:
def get_smoothed_bigram_corpus(unigram_corpus, bigrams):
    df = pd.DataFrame(1, index = unigram_corpus, columns = unigram_corpus) 
    for bigram in bigrams:
        df.loc[bigram[0], bigram[1]] += bigrams[bigram]
    return df

In [103]:
df = pd.DataFrame(1, index =deceptive_unigram, columns =deceptive_unigram) 

In [104]:
get_smoothed_bigram_corpus(deceptive_unigram, deceptive_bigram)

,<s>,I,was,here,on,business,so,needed,to,get,...,recoup,rockin,ROYAL,gifts,allergy-friendly,tree,informal,tranquility,cleaners,beutiful
<s>,1,135,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
I,1,1,326,1,1,1,2,15,5,12,...,1,1,1,1,1,1,1,1,1,1
was,1,3,1,4,19,1,38,1,15,1,...,1,1,1,1,1,1,1,1,1,1
here,1,1,4,1,4,1,1,1,2,1,...,1,1,1,1,1,1,1,1,1,1
on,1,1,1,1,1,15,1,1,2,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tree,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
informal,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
tranquility,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
cleaners,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [105]:
def get_smoothed_bigram_prob(bigram, smoothed_bigram_corpus):
    return smoothed_bigram_corpus.loc[bigram[0], bigram[1]]/df.loc[bigram[0]].sum()

In [106]:
get_smoothed_bigram_prob(('I', 'am'), df)

0.0001684068710003368

## Part 4 – Validation

In [107]:
with open('./DATASET/validation/truthful.txt') as t, open('./DATASET/validation/deceptive.txt') as d:
    truthful_v = t.read()
    deceptive_v =d.read()

In [108]:
# replace all unknown words with <UNK> token
def check_for_unk_words(word_list, corpus):
    for i, word in enumerate(word_list):
        if word not in corpus:
              word_list[i] = '<UNK>'
    return word_list

## Validation Unigrams

In [109]:
# Cleaning & handling unknown words for truthful validation
cleaned_truthful_word_list_v = check_for_unk_words(add_start_characters(truthful_v).split(), truthful_unigram)
truthful_unigram_v = Counter(cleaned_truthful_word_list_v)

# Cleaning & handling unknown words for deceptive validation
cleaned_deceptive_word_list_v = check_for_unk_words(add_start_characters(deceptive_v).split(), deceptive_unigram)
deceptive_unigram_v = Counter(cleaned_deceptive_word_list_v)

## Validation Bigrams

In [110]:
truthful_bigram_v = get_bigram_counts(cleaned_truthful_word_list_v)
deceptive_bigram_v = get_bigram_counts(cleaned_deceptive_word_list_v)

In [113]:
get_smoothed_bigram_corpus(truthful_unigram_v, truthful_bigram_v)

,<s>,I,stayed,for,four,nights,while,attending,a,conference,...,NOTHING,training,opted,own,sterile,lives,indoor,movie,Mag,affordable
<s>,1,28,1,1,1,1,1,1,2,1,...,1,1,1,1,1,1,1,1,1,1
I,1,1,21,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
stayed,1,1,1,3,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
for,1,1,1,1,3,1,1,1,43,1,...,1,1,1,1,1,1,1,1,1,1
four,1,1,1,1,1,3,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
lives,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
indoor,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
movie,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Mag,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


## Perplexity

In [212]:
def get_n_gram_model(wordlist, n):
    # wordlist is an in-order list of the words on which to build the model
    # n determines whether to use unigram or bigram estimation (n must be 1 or 2)
    n_gram_model = {}
    for i, word in enumerate(wordlist):
        if i + n > len(wordlist) - 1:
            break
        key = tuple([wordlist[i+x] for x in range(n)])
        if n == 1:
            # by convention, we now use strings as keys for unigram models and tuples for bigram models
            key = word
        if key not in n_gram_model:
            n_gram_model[key] = 1
        else:
            n_gram_model[key] += 1 
    return n_gram_model

In [213]:
def get_n_gram_prob(sequence, model):
    # model is a dictionary representing the n-grams and counts for some dataset
    
    # this just gets n implied from the model
    for key in model:
        if isinstance(key, str):
            n = 1
        else:
            n = len(key)
        break
        
    if n == 1:
        # case if model uses unigram estimation
        result = 1
        for word in sequence:
            result = result * get_unigram_prob(model, word)
        return result
    elif n == 2:
        # case if model uses bigram estimation
        bigrams = get_bigram_counts(sequence)
        result = 1
        for bigram in bigrams:
            result = result * get_bigram_prob(model, bigram)
        return result
    else:
        # else cannot compute
        return 0

In [214]:
get_n_gram_prob(['I', 'like', 'hotels'], get_n_gram_model(cleaned_truthful_word_list_v, 2))

0.0

In [117]:
import math
def compute_perplexity(N, model, n):
    # N is the number of tokens, so the length of the unigram corpus
    acc = 0

    for i, n_gram in enumerate(model):
        if i == N:
            break
        acc -= math.log(get_n_gram_prob(n_gram, model))
    return math.e ** ((1/N) * acc)

In [119]:
compute_perplexity(len(truthful_unigram_v), get_n_grams(cleaned_truthful_word_list_v, 1), 1)

ValueError: math domain error

## Language Model Classifier

In [ ]:
#review is the unigram for the review
compute_perplexity(review)

## Naïve Bayes Classifier

In [26]:
from sklearn.naive_bayes import GaussianNB

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
nb = GaussianNB()
# y_pred = gnb.fit(iris.data, iris.target).predict(iris.data)

(1,)

('s',)

str